# Import Libraries


In [25]:
import random
import time
import csv

import numpy as np
import pandas as pd

from pprint import pprint as print 

# Define Classes

In [26]:
class Customer:
    def __init__(self, id, lat, lon, demand):
        self.id = id
        self.lat = lat
        self.lon = lon
        self.demand = demand
        
    def distance(self, customer):
        latDis = abs(self.lat - customer.lat)
        longDis = abs(self.lon - customer.lon)
        distance = 100 * np.sqrt((latDis ** 2) + (longDis ** 2))
        return distance
    
    def __repr__(self):
        return "(" + str(self.lat) + "," + str(self.lon) + ")"

In [27]:
class Vehicle:
    def __init__(self, type):
        self.type = type
        self.routes = []
        self.rate = None
        self.capacity = None

    def addRoute(self, route):
        self.routes.append(route)

    def setRateCapacity(self):
        if self.type == 'A':
            self.rate = 1.2
            self.capacity = 25
        elif self.type == 'B':
            self.rate = 1.5
            self.capacity = 30
        else:
            self.rate = 0
            self.capacity = 0
    
    def setStartPoint(self):
        self.routes.append(Customer(-1, 4.4184, 114.0932, 0))

In [28]:
class Fitness:
    def __init__(self, vehicle):
        self.vehicle = vehicle
        self.fitness = None
    
    # fitness is based on cost, since distance is directly in correlation with cost,
    # taking cost can be a more accurate representation of the efficiency of the route

    # routes = customer
    def cost(self):
        routes = self.vehicle.routes
        distance = 0

        for i in range (0, len(routes)):
            fromCity = routes[i]
            if i+1 < len(routes):
                toCity = routes[i+1]

            # return to depot
            else:
                toCity = routes[0]
            
            distance += fromCity.distance(toCity)

        return distance*(self.vehicle.rate)
    
    # def fitness(self):
    #     cost = cost(self)

    #     # give higher ranking for lower cost
    #     return abs(1/cost)
            
            

In [29]:
# fill in routes - choose vehicle - ensure routes is selected based on demands


# Generate Population

In [30]:
data = pd.read_csv('data.csv')
data.head()

li = data.values.tolist()

oriRoutes = []
for i in li:
    oriRoutes.append(Customer(i[0], i[1], i[2], i[3]))

solutions = [] #array of vehicles

for s in range(1000):
    print('iteration'+ str(s+1))
    routes = oriRoutes.copy()

    vehicles = []

    while(len(routes) !=0):
        # Randomly choose a vehicle
        type = random.choice(['A', 'B'])
        vehicle = Vehicle(type)
        vehicle.setStartPoint()
        vehicle.setRateCapacity()

        print('Chosen vehicle '+ vehicle.type)

        # Randomly fill in the routes
        filledCapacity = 0

        while(routes):
            selectedRoute = random.choice(routes)
            if((filledCapacity+selectedRoute.demand) <= vehicle.capacity):
                filledCapacity += selectedRoute.demand
                vehicle.addRoute(selectedRoute)
                routes.remove(selectedRoute)
                print('Route ' + str(selectedRoute.id) + ', demand ' + str(selectedRoute.demand))
            else:
                break
            

        vehicles.append(vehicle)

    solutions.append(vehicles)
    
solutions


'iteration1'
'Chosen vehicle B'
'Route 1.0, demand 5.0'
'Route 3.0, demand 3.0'
'Route 10.0, demand 8.0'
'Route 5.0, demand 5.0'
'Route 7.0, demand 3.0'
'Chosen vehicle B'
'Route 2.0, demand 8.0'
'Route 8.0, demand 6.0'
'Route 6.0, demand 8.0'
'Route 4.0, demand 6.0'
'Chosen vehicle A'
'Route 9.0, demand 5.0'
'iteration2'
'Chosen vehicle B'
'Route 2.0, demand 8.0'
'Route 6.0, demand 8.0'
'Route 5.0, demand 5.0'
'Route 8.0, demand 6.0'
'Route 3.0, demand 3.0'
'Chosen vehicle A'
'Route 7.0, demand 3.0'
'Route 4.0, demand 6.0'
'Route 10.0, demand 8.0'
'Route 9.0, demand 5.0'
'Chosen vehicle A'
'Route 1.0, demand 5.0'
'iteration3'
'Chosen vehicle B'
'Route 8.0, demand 6.0'
'Route 6.0, demand 8.0'
'Route 7.0, demand 3.0'
'Route 5.0, demand 5.0'
'Route 9.0, demand 5.0'
'Route 3.0, demand 3.0'
'Chosen vehicle B'
'Route 4.0, demand 6.0'
'Route 10.0, demand 8.0'
'Route 1.0, demand 5.0'
'Route 2.0, demand 8.0'
'iteration4'
'Chosen vehicle A'
'Route 8.0, demand 6.0'
'Route 4.0, demand 6.0'
'Route

[[<__main__.Vehicle at 0x209ea8a1a30>,
 [<__main__.Vehicle at 0x209eade0470>, <__main__.Vehicle at 0x209eade2e40>],
 [<__main__.Vehicle at 0x209eae114c0>, <__main__.Vehicle at 0x209eae11e80>],
 [<__main__.Vehicle at 0x209eae11370>, <__main__.Vehicle at 0x209eae114f0>],
 [<__main__.Vehicle at 0x209eae11d60>, <__main__.Vehicle at 0x209eae120c0>],
 [<__main__.Vehicle at 0x209eae132c0>, <__main__.Vehicle at 0x209eae12ff0>],
 [<__main__.Vehicle at 0x209eae13cb0>, <__main__.Vehicle at 0x209eae134a0>],
 [<__main__.Vehicle at 0x209ebe38200>, <__main__.Vehicle at 0x209ebe39070>],
 [<__main__.Vehicle at 0x209ebe38c80>, <__main__.Vehicle at 0x209ebe38bf0>],
 [<__main__.Vehicle at 0x209ebe39100>, <__main__.Vehicle at 0x209ebe39760>],
 [<__main__.Vehicle at 0x209ebe39ee0>, <__main__.Vehicle at 0x209ebe393d0>],
 [<__main__.Vehicle at 0x209ebe3a480>, <__main__.Vehicle at 0x209ebe3a1e0>],
 [<__main__.Vehicle at 0x209ebe3a5a0>, <__main__.Vehicle at 0x209ebe3a690>],
 [<__main__.Vehicle at 0x209ebe3a3f0>

In [31]:
# routes do not need to include depot at end, but shoudl include depot at start

# Tournament Selection

In [32]:
# Randomly choose 20 samples, get one parent
def getParent():
    choices = random.sample(solutions, 20)

    chosenSolutionCost = []
    for vehicles in choices:
        solutionCost = 0
        for vehicle in vehicles:
            fitness = Fitness(vehicle)
            cost = fitness.cost()
            solutionCost += cost
        chosenSolutionCost.append(solutionCost)


    minpos = chosenSolutionCost.index(min(chosenSolutionCost))

    return choices[minpos]

firstParent = getParent()
secondParent = getParent()

print(firstParent)
secondParent


# Crossover

In [33]:
def findMissing(li):
    return sorted(set(range(1, 11)).difference(li))


def convertToOneDList(li):
    convertedList = []
    for i in range(len(li)):
        for j in range(len(li[i])):
            convertedList.append(int(li[i][j]))
    return convertedList

def getDuplicatedElements(li):
    duplicates = []
    for x in li:
        if x not in duplicates and li.count(x) >1:
            duplicates.append(x)
        
    return duplicates


In [61]:
# drop start point when crossing over

first = [[3.0, 4.0, 2.0, 6.0], [7.0, 1.0, 5.0, 9.0], [8.0, 10.0]]
firstVehicles = ['B', 'A', 'B']
second = [[4.0, 6.0, 5.0], [3.0, 1.0, 2.0, 7.0, 10.0], [8.0, 9.0]]
secondVehicles = ['A', 'B', 'A']


selectedVehicleIdx = 1
selectedToSwapVehicle = [3.0, 1.0, 2.0, 7.0, 10.0]
selectedToSwapVehicle2 = [7.0, 1.0, 5.0, 9.0]
# first = [[],[],[]]
# firstVehicles = []

# for i in range(len(firstParent)):
#     firstParent[i].routes.pop(0)
#     firstVehicles.append(firstParent[i].type)
#     # for every route
#     for j in range(len(firstParent[i].routes)):
#         first[i].append(firstParent[i].routes[j].id)



# second = [[],[],[]]
# secondVehicles = []


# for i in range(len(secondParent)):
#     secondParent[i].routes.pop(0)
#     secondVehicles.append(secondParent[i].type)
#     # for every route
#     for j in range(len(secondParent[i].routes)):
#         second[i].append(secondParent[i].routes[j].id)


# selectedFirstVehicle = random.choice(first)
# selectedFirstVehicleIdx = first.index(selectedFirstVehicle)

# selectedSecondVehicle = random.choice(second)
# selectedSecondVehicleIdx = second.index(selectedSecondVehicle)



# Random Choice of 1,2,3,4
choice = random.randint(1, 4)
# Case 1 - first parent is  main, two vehicles remain, one vehicle changes
print(choice)
if (choice == 1 or choice == 2):
    if (choice == 1):
        parent = first.copy()
        # selectedVehicleIdx = random.randInt(0, len(parent)-1)
        # selectedToSwapVehicle = random.choice(secondary)
        parent[selectedVehicleIdx] = selectedToSwapVehicle
        parent = convertToOneDList(parent)

    # Case 2 - second parent is main, two vehicles remain, one vehicle changes
    elif (choice == 2):
        parent = second.copy()
        # selectedVehicleIdx = random.randInt(0, len(parent)-1)
        # selectedToSwapVehicle = random.choice(secondary)
        parent[selectedVehicleIdx] = selectedToSwapVehicle2
        parent = convertToOneDList(parent)

    print(parent)
    missing = findMissing(parent)
    duplicates = getDuplicatedElements(parent)

    child = []
    for i in parent:
        if i not in child:
            child.append(i)
        else:
            if(missing):
                child.append(missing[0])
                missing.pop(0)

    if (missing):
        child += missing


elif (choice == 3 or choice == 4):
    # Case 3 - take first parent as a whole, no crossover, direcly sent to mutation
    if (choice == 3):
        child = convertToOneDList(first)
    # Case 4 - take second parent as a whole, no crossover, direcly sent to mutation
    elif (choice == 4):
        child = convertToOneDList(second)

print(child)


2
[4, 6, 5, 7, 1, 5, 9, 8, 9]
[4, 6, 5, 7, 1, 2, 9, 8, 3, 10]


# Mutation

In [ ]:
# Vehicle Mutation
def vehicleMutation(li):
    idx = random.randint(0, len(li)-1)
    if li[idx] == 'A':
        li[idx] = 'B'
    else:
        li[idx] = 'A'
    
    return li

# Routes Mutation - Swap
def routeMutation(li):
    aIdx = random.randint(0, len(li)-1)
    bIdx = random.randint(0, len(li)-1)

    temp = li[aIdx]
    li[aIdx] = li[bIdx]
    li[bIdx] = temp

    return li

In [ ]:
print(firstVehicles)
print(vehicleMutation(firstVehicles))
print(firstVehicles)

print(first)
print(routeMutation(first))

['B', 'B', 'B']
['B', 'A', 'B']
['B', 'A', 'B']
[3, 4, 7, 6, 5, 1, 9, 2, 10, 8]
[3, 4, 7, 6, 5, 1, 9, 2, 10, 8]
